In [2]:
import numpy as np
import tensorflow as tf

import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

import optuna
print(optuna.__version__)

3.6.1


c:\Users\Kuba\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
Features=pd.read_csv("../data/data_proccesed.csv")
X = Features.iloc[: ,:-1].values
Y = Features['labels'].values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

def custom_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.02))
scheduler = LearningRateScheduler(custom_scheduler)

model_save = ModelCheckpoint(r"../model/best_model.h5",
                              save_best_only=True,
                             mode='max',
                             monitor='val_accuracy',
                             initial_value_threshold=0.80,
                             verbose=1)


normalizer = layers.Normalization()
normalizer.adapt(X)

In [7]:
def objective(trial):
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.4)
    batch_size = trial.suggest_categorical('batch_size', [32, 64])

    n_1_layers = trial.suggest_int('n_layers', 1, 3)
    n_2_layers = trial.suggest_int('n_layers', 1, 3)
    n_layers_dense = trial.suggest_int('n_layers', 1, 3)

    model = models.Sequential()
    model.add(layers.Input(shape=(x_train.shape[1],)))
    model.add(normalizer)
    model.add(layers.Reshape((x_train.shape[1], 1))) 

    for i in range(n_1_layers):
        num_neurons_conv_1 = trial.suggest_int(f'n_1_units_l{i}', 128, 1024, log=True)
        model.add(tf.keras.layers.Dense(num_neurons_conv_1, activation='relu')),

    model.add(layers.MaxPooling1D())

    for i in range(n_2_layers):
        num_neurons_conv_2 = trial.suggest_int(f'n_2_units_l{i}', 128, 1024, log=True)
        model.add(tf.keras.layers.Dense(num_neurons_conv_2, activation='relu')),
    model.add(layers.MaxPooling1D())

    model.add(layers.Flatten())
        
    for i in range(n_layers_dense):
        num_neurons_dense = trial.suggest_int(f'n_3_units_l{i}', 128, 1024, log=True)
        layers.Dropout(dropout_rate),
        model.add(tf.keras.layers.Dense(num_neurons_dense, activation='relu')),

    model.add(layers.Dense(9, activation='softmax'))
    
    

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'],
    )
    
    # Fit the model
    history = model.fit(
        x_train, y_train,
        epochs=60, batch_size=batch_size,
        validation_data=(x_test, y_test),
        callbacks=[scheduler, model_save],
        verbose=0
    )
    
    # Return the validation loss
    return history.history['val_loss'][-1]

# # Perform optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)

# Print best parameters
print('Best trial:')
trial = study.best_trial
print('  Value: {}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))